In [ ]:
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 6.9 MB/s 


In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import display
import matplotlib.pyplot as plt
import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings; warnings.filterwarnings('ignore')
import cv2
import pydicom
#from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score

from pylab import rcParams
rcParams['figure.figsize'] = 20,5

In [ ]:
!gdown --id 1diztKyKFfhINpY-8R_KB84kV8akW2NjD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1diztKyKFfhINpY-8R_KB84kV8akW2NjD
To: /content/test_train_val.zip
100% 480M/480M [00:10<00:00, 47.7MB/s]


In [ ]:
!unzip -qq test_train_val.zip

In [ ]:
with open('test_train_val/train_meta.csv') as f:
  train_df= pd.read_csv(f)
with open('test_train_val/train_signal.csv') as f:
  train_signal = pd.read_csv(f)

with open('test_train_val/valid_meta.csv') as f:
  valid_df= pd.read_csv(f)
with open('test_train_val/valid_signal.csv') as f:
  valid_signal = pd.read_csv(f)

with open('test_train_val/test_meta.csv') as f:
  test_df= pd.read_csv(f)
with open('test_train_val/test_signal.csv') as f:
  test_signal = pd.read_csv(f)

print(train_df.shape)

train_df

(17441, 37)


,ecg_id,age,sex,height,weight,nurse,site,device,NORM,MI,...,sub_ISC_,sub_SEHYP,sub_ISCI,sub_CRBBB,sub_CLBBB,sub_LAO/LAE,sub_ILBBB,sub_WPW,sub_PMI,strat_fold
0,1,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,3
1,2,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,2
2,3,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,5
3,4,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,3
4,5,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17436,21832,63.0,0,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,7
17437,21833,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,7
17438,21834,93.0,0,NaN,NaN,1.0,2.0,AT-60 3,1,0,...,0,0,0,0,0,0,0,0,0,4
17439,21835,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
class PTBXLDatasetPreprocesser():
    def __init__(self):
        pass
    
    def save(self, filename):
        data = {
            'superclass_cols': self.superclass_cols,
            'subclass_cols': self.subclass_cols,
            'meta_num_cols': self.meta_num_cols,
            'meta_num_means': self.meta_num_means,
            'min_max_scaler': self.min_max_scaler,
            'meta_cat_cols': self.meta_cat_cols,
            'cat_lablers': self.cat_lablers,
            'bclass_cols': self.bclass_cols,
            
        }
        pd.to_pickle(data, filename)
        
    def load(self, filename):
        data = pd.read_pickle(filename)
        self.min_max_scaler = data['min_max_scaler']
        self.cat_lablers = data['cat_lablers']
        #self.binary_lablers = data['binary_lablers']
        
    def fit(self, x, y):
        x = x.copy()
        y = y.copy()
        
        self.superclass_cols = [ 'MI', 'STTC', 'CD', 'HYP']
        
        self.subclass_cols = [col for col in y.columns if 'sub_' in col]

        self.bclass_cols = ['NORM']
        
        self.meta_num_cols = ['age', 'height', 'weight']
        self.meta_num_means = []
        for col in self.meta_num_cols:
            print(col, y[col].mean())
            y[col] = y[col].fillna(y[col].mean())
            self.meta_num_means += [y[col].mean()]
            
        self.min_max_scaler = MinMaxScaler().fit(y[self.meta_num_cols])
        
        self.meta_cat_cols = ['sex'] #, 'nurse', 'device']
        self.cat_lablers = [LabelEncoder().fit(y[col].fillna('none').astype(str)) for col in self.meta_cat_cols]
        return self

        #self.min_max_scaler = MinMaxScaler().fit(y[self.meta_num_cols])
        
        #self.meta_binary_cols = ['NORM'] 
        #self.binary_lablers = [LabelEncoder().fit(y[col].fillna('none').astype(str)) for col in self.meta_binary_cols]
        #return self
    
    def transform(self, x, y):
        
        channel_cols = x.columns.tolist()[1:]
        
        ret = []
        x = x[channel_cols].values.reshape(-1, 1000, 12)
        print(x.shape)
        ret += [x] # signal
        
        y_ = y.copy()
        
        for i, col in enumerate(self.meta_num_cols):
            y_[col] = y_[col].fillna(self.meta_num_means[i])
        y_[self.meta_num_cols] = self.min_max_scaler.transform(y_[self.meta_num_cols])
        y_[self.meta_num_cols] = np.clip(y_[self.meta_num_cols], 0., 1.) # prevent extreme value far from train set
        
        ret += [y_[self.meta_num_cols]] # meta num features
        
        for i, col in enumerate(self.meta_cat_cols):
            y_[col] = y_[col].fillna('none').astype(str)
            y_[col] = self.cat_lablers[i].transform(y_[col]) 
        
        ret += [y_[self.meta_cat_cols]] # meta cat features

        #for i, col in enumerate(self.meta_binary_cols):
           # y_[col] = y_[col].fillna('none').astype(str)
           # y_[col] = self.binary_lablers[i].transform(y_[col]) 
        
        #ret += [y_[self.meta_binary_cols]] # binary class target
        
        if np.isin(self.superclass_cols, y.columns).sum() == len(self.superclass_cols):
            ret += [y[self.superclass_cols].fillna(0).astype(int)] # superclass targets
        
        if np.isin(self.subclass_cols, y.columns).sum() == len(self.subclass_cols):
            ret += [y[self.subclass_cols].fillna(0).astype(int)] # subclass targets

        if np.isin(self.bclass_cols, y.columns).sum() == len(self.bclass_cols):
            ret += [y[self.bclass_cols].fillna(0).astype(int)]
        
        return ret

In [ ]:
data_preprocessor = PTBXLDatasetPreprocesser()
data_preprocessor.fit(train_signal, train_df)
train_signal, train_meta_num_feats, train_meta_cat_feats, train_superclass, train_subclass, train_bclass = data_preprocessor.transform(train_signal, train_df)
valid_signal, valid_meta_num_feats, valid_meta_cat_feats, valid_superclass, valid_subclass, valid_bclass = data_preprocessor.transform(valid_signal, valid_df)
test_signal, test_meta_num_feats, test_meta_cat_feats, test_superclass, test_subclass, test_bclass = data_preprocessor.transform(test_signal, test_df)

print(train_signal.shape)
print(valid_meta_num_feats.isna().sum(), valid_meta_cat_feats.isna().sum(), valid_superclass.isna().sum(), valid_subclass.isna().sum(), valid_bclass.isna().sum())

display(train_meta_num_feats)
display(train_meta_cat_feats)
display(train_superclass)
display(train_subclass)
display(train_bclass)

age 59.63709074169975
height 166.68908469699693
weight 70.69599447513812
(17441, 1000, 12)
(2193, 1000, 12)
(2203, 1000, 12)
(17441, 1000, 12)
age       0
height    0
weight    0
dtype: int64 sex    0
dtype: int64 MI      0
STTC    0
CD      0
HYP     0
dtype: int64 sub_NORM         0
sub_IMI          0
sub_STTC         0
sub_NST_         0
sub_LVH          0
sub_LAFB/LPFB    0
sub_RVH          0
sub_RAO/RAE      0
sub_IRBBB        0
sub_IVCD         0
sub_LMI          0
sub_AMI          0
sub__AVB         0
sub_ISCA         0
sub_ISC_         0
sub_SEHYP        0
sub_ISCI         0
sub_CRBBB        0
sub_CLBBB        0
sub_LAO/LAE      0
sub_ILBBB        0
sub_WPW          0
sub_PMI          0
dtype: int64 NORM    0
dtype: int64


,age,height,weight
0,0.580645,0.791572,0.282927
1,0.182796,0.791572,0.317073
2,0.376344,0.791572,0.312195
3,0.236559,0.791572,0.375610
4,0.182796,0.791572,0.317073
...,...,...,...
17436,0.655914,0.791572,0.320468
17437,0.698925,0.791572,0.320468
17438,0.978495,0.791572,0.320468
17439,0.612903,0.791572,0.320468


,sex
0,1
1,0
2,1
3,0
4,1
...,...
17436,0
17437,1
17438,0
17439,1


,MI,STTC,CD,HYP
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
17436,0,0,1,0
17437,0,1,0,0
17438,0,0,0,0
17439,0,1,0,0


,sub_NORM,sub_IMI,sub_STTC,sub_NST_,sub_LVH,sub_LAFB/LPFB,sub_RVH,sub_RAO/RAE,sub_IRBBB,sub_IVCD,...,sub_ISCA,sub_ISC_,sub_SEHYP,sub_ISCI,sub_CRBBB,sub_CLBBB,sub_LAO/LAE,sub_ILBBB,sub_WPW,sub_PMI
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17436,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
17437,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17438,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17439,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


,NORM
0,1
1,1
2,1
3,1
4,1
...,...
17436,0
17437,0
17438,1
17439,0


In [ ]:
signal=np.array(train_signal)
signal.shape
#print(signal)

(17441, 1000, 12)

In [ ]:
# signal1 = np.reshape(signal(17441,10,1200))

In [ ]:
signal[0]

array([[-0.119, -0.055,  0.064, ..., -0.026, -0.039, -0.079],
       [-0.116, -0.051,  0.065, ..., -0.031, -0.034, -0.074],
       [-0.12 , -0.044,  0.076, ..., -0.028, -0.029, -0.069],
       ...,
       [ 0.069,  0.   , -0.069, ...,  0.024, -0.041, -0.058],
       [ 0.086,  0.004, -0.081, ...,  0.242, -0.046, -0.098],
       [ 0.022, -0.031, -0.054, ...,  0.143, -0.035, -0.12 ]])

In [ ]:
target=np.array(train_bclass)
target.shape

(17441, 1)

In [ ]:
v_signal=np.array(valid_signal)

In [ ]:
v_signal.shape

(2193, 1000, 12)

In [ ]:
v_target=np.array(valid_bclass)
v_target.shape

(2193, 1)

In [ ]:
!pip install ecg-plot
#import physionet_challenge_utility_script as pc
import ecg_plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
#from keras.utils import plot_model
#from keras.preprocessing.sequence import pad_sequences
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPool1D, ZeroPadding1D, LSTM, Bidirectional, ConvLSTM1D
from keras.models import Sequential, Model
#from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, auc
from keras.layers import concatenate, Flatten
from scipy import optimize
from scipy.io import loadmat
import os
%load_ext autoreload
%autoreload
%reload_ext autoreload

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
vot = []

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_AUC', factor=0.1, patience=3
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=11)

**alexnet**

In [ ]:
alexNet_model=Sequential()

alexNet_model.add(Conv1D(filters=96, kernel_size=11, strides=4, input_shape=(1000,12)))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(Conv1D(filters=256, kernel_size=5, padding='same'))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(Conv1D(filters=384, padding='same', kernel_size=3))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(Conv1D(filters=384, kernel_size=3))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(Conv1D(filters=256, kernel_size=3))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(GlobalAveragePooling1D())
alexNet_model.add(Dense(128, activation='relu'))
alexNet_model.add(Dropout(0.4))
alexNet_model.add(Dense(128, activation='relu'))
alexNet_model.add(Dropout(0.4))
alexNet_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# alexNet_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')
alexNet_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
        name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
    )])

In [ ]:
alexNet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 248, 96)           12768     
                                                                 
 batch_normalization (BatchN  (None, 248, 96)          384       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 248, 96)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 124, 96)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 124, 256)          123136    
                                                                 
 batch_normalization_1 (Batc  (None, 124, 256)         1

In [ ]:
batchsize = 32
alexNet_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 33s 25ms/step - loss: 0.3838 - accuracy: 0.8265 - Recall: 0.8307 - Precision: 0.7843 - AUC: 0.9064 - val_loss: 0.3675 - val_accuracy: 0.8431 - val_Recall: 0.8903 - val_Precision: 0.7809 - val_AUC: 0.9185 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 8s 15ms/step - loss: 0.3348 - accuracy: 0.8584 - Recall: 0.8696 - Precision: 0.8174 - AUC: 0.9292 - val_loss: 0.4324 - val_accuracy: 0.8057 - val_Recall: 0.9697 - val_Precision: 0.7004 - val_AUC: 0.9262 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 8s 15ms/step - loss: 0.3213 - accuracy: 0.8603 - Recall: 0.8693 - Precision: 0.8209 - AUC: 0.9351 - val_loss: 0.3209 - val_accuracy: 0.8605 - val_Recall: 0.9039 - val_Precision: 0.8017 - val_AUC: 0.9338 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 8s 15ms/step - loss: 0.3073 - accuracy: 0.8711 - Recall: 0.8721 - Precision: 0.8387 - AUC: 0.9410 - val_loss: 0.3126 - val_accuracy: 0.

In [ ]:
y_pred=alexNet_model.predict(test_signal)
predict_a=[int(x+0.5) for x in y_pred]
vot.append(np.array(predict_a))
conf_mat=confusion_matrix(test_bclass,predict_a)
print(conf_mat)

[[1027  212]
 [  90  874]]


In [ ]:
TP= conf_mat[0,0]
FP= conf_mat[0,1]
FN= conf_mat[1,0]
TN= conf_mat[1,1]
print('FP:'+str(FP))
print('TP:'+str(TP))
print('FN:'+str(FN))
print('TN:'+str(TN))

FNR = FN/(TP+FN)
TPR = TP/(TP+FN)

FP:212
TP:1027
FN:90
TN:874


In [ ]:
# auc(TPR,FNR)

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_bclass, y_pred)
metrics.auc(fpr, tpr)

0.942274588997284

In [ ]:
# tpr

In [ ]:
accuracy_score(test_bclass,predict_a)

0.8629142078983205

In [ ]:
# auc(test_bclass,predict_a)

In [ ]:
# roc_auc_score(test_bclass,predict_a)

**vgg**

In [ ]:
vgg_16_model=Sequential()

vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same',  input_shape=(1000,12)))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(GlobalAveragePooling1D())
vgg_16_model.add(Dense(256, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(128, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(1, activation='sigmoid'))

In [ ]:
vgg_16_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
        name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
    )])

In [ ]:
batchsize = 32
vgg_16_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 42s 69ms/step - loss: 0.3969 - accuracy: 0.8307 - Recall: 0.8240 - Precision: 0.7952 - AUC: 0.9022 - val_loss: 0.3860 - val_accuracy: 0.8431 - val_Recall: 0.9039 - val_Precision: 0.7744 - val_AUC: 0.9251 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 36s 67ms/step - loss: 0.3419 - accuracy: 0.8556 - Recall: 0.8572 - Precision: 0.8198 - AUC: 0.9269 - val_loss: 0.3584 - val_accuracy: 0.8431 - val_Recall: 0.9060 - val_Precision: 0.7734 - val_AUC: 0.9221 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 37s 67ms/step - loss: 0.3262 - accuracy: 0.8630 - Recall: 0.8726 - Precision: 0.8238 - AUC: 0.9341 - val_loss: 0.3442 - val_accuracy: 0.8377 - val_Recall: 0.7659 - val_Precision: 0.8474 - val_AUC: 0.9302 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 37s 67ms/step - loss: 0.3094 - accuracy: 0.8706 - Recall: 0.8764 - Precision: 0.8350 - AUC: 0.9401 - val_loss: 0.6052 - val_accuracy:

In [ ]:
y_pred=vgg_16_model.predict(test_signal)
predict_b=[int(x+0.5) for x in y_pred]
vot.append(np.array(predict_b))
conf_mat=confusion_matrix(test_bclass,predict_b)
print(conf_mat)

[[1034  205]
 [  79  885]]


In [ ]:
accuracy_score(test_bclass,predict_b)

0.8710848842487517

In [ ]:
# roc_auc_score(test_bclass,predict_b)

In [ ]:
# auc(test_bclass,predict_b)
# from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_bclass, y_pred)
metrics.auc(fpr, tpr)

0.9448332043978713

**inseption**

In [ ]:
def inception_block(prev_layer):
    
    conv1=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv1=BatchNormalization()(conv1)
    conv1=Activation('relu')(conv1)
    
    conv3=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
    conv3=Conv1D(filters = 64, kernel_size = 3, padding = 'same')(conv3)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
    
    conv5=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    conv5=Conv1D(filters = 64, kernel_size = 5, padding = 'same')(conv5)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    
    pool= MaxPool1D(pool_size=3, strides=1, padding='same')(prev_layer)
    convmax=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(pool)
    convmax=BatchNormalization()(convmax)
    convmax=Activation('relu')(convmax)
    
    layer_out = concatenate([conv1, conv3, conv5, convmax], axis=1)
    
    return layer_out

def inception_model(input_shape):
    X_input=Input(input_shape)
    
    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(filters = 64, kernel_size = 7, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPool1D(pool_size=3, strides=2, padding='same')(X)
    
    X = Conv1D(filters = 64, kernel_size = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = inception_block(X)
    X = inception_block(X)
    
    X = MaxPool1D(pool_size=7, strides=2, padding='same')(X)
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name='Inception')
    
    return model

In [ ]:
inception_model = inception_model(input_shape = (1000,12))

In [ ]:
inception_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
        name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
    )])

In [ ]:
batchsize = 32
inception_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 40s 65ms/step - loss: 0.3696 - accuracy: 0.8326 - Recall: 0.8323 - Precision: 0.7939 - AUC: 0.9129 - val_loss: 0.3757 - val_accuracy: 0.8372 - val_Recall: 0.9363 - val_Precision: 0.7517 - val_AUC: 0.9240 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 34s 63ms/step - loss: 0.3208 - accuracy: 0.8599 - Recall: 0.8612 - Precision: 0.8252 - AUC: 0.9349 - val_loss: 0.3281 - val_accuracy: 0.8541 - val_Recall: 0.8997 - val_Precision: 0.7935 - val_AUC: 0.9305 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 35s 63ms/step - loss: 0.3069 - accuracy: 0.8691 - Recall: 0.8720 - Precision: 0.8352 - AUC: 0.9409 - val_loss: 0.3176 - val_accuracy: 0.8614 - val_Recall: 0.9091 - val_Precision: 0.8004 - val_AUC: 0.9365 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 35s 63ms/step - loss: 0.2933 - accuracy: 0.8736 - Recall: 0.8783 - Precision: 0.8394 - AUC: 0.9460 - val_loss: 0.3104 - val_accuracy:

In [ ]:
y_pred=inception_model.predict(test_signal)
predict_c=[int(x+0.5) for x in y_pred]
vot.append(np.array(predict_c))
conf_mat=confusion_matrix(test_bclass,predict_c)
print(conf_mat)

[[1022  217]
 [  94  870]]


In [ ]:
accuracy_score(test_bclass,predict_c)

0.8588288697231049

In [ ]:
# roc_auc_score(test_bclass,predict_c)

In [ ]:
# auc(test_bclass,predict_c)
# from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_bclass, y_pred)
metrics.auc(fpr, tpr)

0.9419212723418365

**Resnet**

In [ ]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv1D(filters = F1, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(filters = F2, kernel_size = f, activation='relu', strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(filters = F3, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)

    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def convolutional_block(X, f, filters, s = 2):
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv1D(F1, 1, activation='relu', strides = s)(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(F2, f, activation='relu', strides = 1,padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(F3, 1, strides = 1)(X)
    X = BatchNormalization()(X)

    X_shortcut = Conv1D(F3, 1, strides = s)(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def ResNet50(input_shape):
    
    X_input = Input(input_shape)

    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(64, 7, strides = 2)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
resNet50_model = ResNet50(input_shape = (1000,12))

In [ ]:
resNet50_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
        name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
    )])

In [ ]:
# resNet50_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy','AUC'])

In [ ]:
resNet50_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1000, 12)]   0           []                               
                                                                                                  
 zero_padding1d_1 (ZeroPadding1  (None, 1006, 12)    0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1d_32 (Conv1D)             (None, 500, 64)      5440        ['zero_padding1d_1[0][0]']       
                                                                                                  
 batch_normalization_32 (BatchN  (None, 500, 64)     256         ['conv1d_32[0][0]']       

In [ ]:
batchsize = 32
resNet50_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 93s 144ms/step - loss: 0.6227 - accuracy: 0.7238 - Recall: 0.6863 - Precision: 0.6822 - AUC: 0.7960 - val_loss: 0.6300 - val_accuracy: 0.7793 - val_Recall: 0.8809 - val_Precision: 0.6950 - val_AUC: 0.8574 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 74s 136ms/step - loss: 0.4263 - accuracy: 0.8077 - Recall: 0.7922 - Precision: 0.7727 - AUC: 0.8887 - val_loss: 0.3837 - val_accuracy: 0.8153 - val_Recall: 0.7753 - val_Precision: 0.7961 - val_AUC: 0.9083 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 74s 136ms/step - loss: 0.3689 - accuracy: 0.8346 - Recall: 0.8236 - Precision: 0.8024 - AUC: 0.9141 - val_loss: 0.8565 - val_accuracy: 0.7893 - val_Recall: 0.9363 - val_Precision: 0.6908 - val_AUC: 0.8814 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 74s 135ms/step - loss: 0.3653 - accuracy: 0.8397 - Recall: 0.8333 - Precision: 0.8059 - AUC: 0.9185 - val_loss: 0.5203 - val_accur

In [ ]:
y_pred=resNet50_model.predict(test_signal)
predict_e=[int(x+0.5) for x in y_pred]
vot.append(np.array(predict_e))
conf_mat=confusion_matrix(test_bclass,predict_e)
print(conf_mat)

[[969 270]
 [106 858]]


In [ ]:
accuracy_score(test_bclass,predict_e)

0.8293236495687698

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(test_bclass, y_pred)
metrics.auc(fpr, tpr)

0.9136835689335865

**cnn**

In [ ]:
cnn_c_model=Sequential()

cnn_c_model.add(Conv1D(filters=5, kernel_size=29, padding='same', input_shape=(1000,12), activation='relu'))
cnn_c_model.add(BatchNormalization())
# cnn_c_model.add(Activation('relu'))
cnn_c_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

cnn_c_model.add(Conv1D(filters=5, kernel_size=15, padding='same', activation='relu'))
cnn_c_model.add(BatchNormalization())
# cnn_c_model.add(Activation('relu'))
cnn_c_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

cnn_c_model.add(Conv1D(filters=5, kernel_size=13,padding='same', activation='relu'))
cnn_c_model.add(BatchNormalization())
# cnn_c_model.add(Activation('relu'))
cnn_c_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

cnn_c_model.add(Conv1D(filters=10, kernel_size=5, padding='same', activation='relu'))
cnn_c_model.add(BatchNormalization())
# cnn_c_model.add(Activation('relu'))
cnn_c_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))


cnn_c_model.add(Conv1D(filters=10, kernel_size=4, padding='same', activation='relu'))
cnn_c_model.add(BatchNormalization())
# cnn_c_model.add(Activation('relu'))
cnn_c_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))


cnn_c_model.add(Conv1D(filters=10, kernel_size=4, padding='same', activation='relu'))
cnn_c_model.add(BatchNormalization())
# cnn_c_model.add(Activation('relu'))
cnn_c_model.add(MaxPool1D(pool_size=3, strides=2, padding='same'))


cnn_c_model.add(Conv1D(filters=10,  kernel_size=3, padding='same', activation='relu'))
cnn_c_model.add(BatchNormalization())
# cnn_c_model.add(Activation('relu'))
cnn_c_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))



cnn_c_model.add(GlobalAveragePooling1D())

cnn_c_model.add(Flatten(data_format=None))

cnn_c_model.add(Dense(10, activation='relu'))

cnn_c_model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
cnn_c_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=[tf.keras.metrics.BinaryAccuracy(
        name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,
   )])

In [ ]:
cnn_c_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_85 (Conv1D)          (None, 1000, 5)           1745      
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 500, 5)           0         
 g1D)                                                            
                                                                 
 conv1d_86 (Conv1D)          (None, 500, 5)            380       
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 250, 5)           0         
 g1D)                                                            
                                                                 
 conv1d_87 (Conv1D)          (None, 250, 5)            330       
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 125, 5)          

In [ ]:
batchsize = 32
cnn_c_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop], validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 8s 11ms/step - loss: 0.6115 - accuracy: 0.5722 - Recall: 0.0748 - Precision: 0.5736 - AUC: 0.6907 - val_loss: 0.5271 - val_accuracy: 0.7314 - val_Recall: 0.5319 - val_Precision: 0.7831 - val_AUC: 0.8508 - lr: 1.0000e-04
Epoch 2/50
546/546 [==============================] - 5s 9ms/step - loss: 0.4944 - accuracy: 0.7865 - Recall: 0.6987 - Precision: 0.7878 - AUC: 0.8690 - val_loss: 0.4647 - val_accuracy: 0.8071 - val_Recall: 0.8150 - val_Precision: 0.7602 - val_AUC: 0.8762 - lr: 1.0000e-04
Epoch 3/50
546/546 [==============================] - 5s 10ms/step - loss: 0.4262 - accuracy: 0.8173 - Recall: 0.8525 - Precision: 0.7585 - AUC: 0.8843 - val_loss: 0.4124 - val_accuracy: 0.8144 - val_Recall: 0.8840 - val_Precision: 0.7408 - val_AUC: 0.8887 - lr: 1.0000e-04
Epoch 4/50
546/546 [==============================] - 5s 10ms/step - loss: 0.3881 - accuracy: 0.8301 - Recall: 0.8657 - Precision: 0.7723 - AUC: 0.9027 - val_loss: 0.4100 - val_ac

In [ ]:
y_pred=cnn_c_model.predict(test_signal)
predict_d=[int (x+0.5) for x in y_pred]
vot.append(np.array(predict_d))
conf_mat=confusion_matrix(test_bclass,predict_d)
print(conf_mat)

[[939 300]
 [108 856]]


In [ ]:
accuracy_score(test_bclass,predict_d)

0.8147980027235587

In [ ]:
# roc_auc_score(test_bclass,predict_d)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(test_bclass, y_pred)
metrics.auc(fpr, tpr)

0.8957874942648837

In [ ]:
# auc(test_bclass,predict_d)

**Ensemble**

In [ ]:
y_prob = np.array(vot).transpose()

In [ ]:
y_prob.shape

(2203, 5)

In [ ]:
import collections

In [ ]:
final_out=[]
for i in range(0, y_prob.shape[0]):
  a=np.array(y_prob[i,:])
  a= collections.Counter(a).most_common()[0][0]
  final_out.append(a)

In [ ]:
accuracy_score(test_bclass,final_out)

0.8624602814344077

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(test_bclass, final_out)
metrics.auc(fpr, tpr)

0.8690898998322165

In [ ]:
print(classification_report(test_bclass,final_out))

'              precision    recall  f1-score   support\n\n           0       0.93      0.82      0.87      1239\n           1       0.80      0.92      0.85       964\n\n    accuracy                           0.86      2203\n   macro avg       0.86      0.87      0.86      2203\nweighted avg       0.87      0.86      0.86      2203\n'